#sample brocker with


""" mosquitto_sub -h 127.0.0.1 -p 1883 -t test-topic -v """

test
"""mosquitto_pub -h 127.0.0.1 -t test-topic -m '{"ping":"pong"}'"""

In [1]:
#python3 -m pip install --user "paho-mqtt>=2.0"

In [2]:
#!/usr/bin/env python3

import threading
import time
import cv2
from bbox_display import TrackerScene
from utils_deepstream import parse_ds_payload
from config import BrokerConfig as cfg
from utils_brocker_mqtt import MqttSubscriber

FPS = 60
WINDOW_NAME = "BBox Viewer"

scene = TrackerScene()

# Thread: רענון חלון כל הזמן – לא תלוי בהודעות
def draw_loop():
    delay = 1.0 / FPS
    while True:
        frame = scene.draw_scene()
        cv2.imshow(WINDOW_NAME, frame)
        key = cv2.waitKey(int(delay * 1000))
        if key == 27:  # ESC
            print("[EXIT] ESC pressed")
            break
    cv2.destroyAllWindows()
    exit(0)

# Callback MQTT: רק עדכון מופעים
def on_message(client, userdata, msg):
    try:
        objs = parse_ds_payload(msg.payload)
        ts = None
        if isinstance(objs, list) and objs:
            ts_str = objs[0].get("@timestamp")
            if ts_str:
                try:
                    ts = float(ts_str)
                except:
                    pass
        now_ts = ts if ts is not None else time.time()
        print(f"[MQTT] {msg.topic}: {len(objs)} obj(s) at {now_ts:.2f}")
        if objs:
            scene.update_from_objs(objs, now_ts if ts else None)  # רק אם יש timestamp
    except Exception as e:
        print(f"[ERROR] in on_message: {e}")

if __name__ == "__main__":
    # Thread ציור
    threading.Thread(target=draw_loop, daemon=True).start()

    # MQTT התחברות
    client = MqttSubscriber(cfg(), on_message=on_message).connect()
    try:
        client.start_forever()
    except KeyboardInterrupt:
        print("\n[MQTT] Stopped by user.")
    finally:
        client.stop()


[MQTT] Connecting to 127.0.0.1:1883 … (topics=['test-topic'])
[MQTT] on_connect: Connected
[MQTT] Subscribed to 'test-topic' (qos=0)
[DRAW] Frame with 0 object(s) at 1757442668.07
[DRAW] Frame with 0 object(s) at 1757442668.13
[DRAW] Frame with 0 object(s) at 1757442668.14
[DRAW] Frame with 0 object(s) at 1757442668.16
[DRAW] Frame with 0 object(s) at 1757442668.18
[DRAW] Frame with 0 object(s) at 1757442668.20
[DRAW] Frame with 0 object(s) at 1757442668.21
[DRAW] Frame with 0 object(s) at 1757442668.23
[DRAW] Frame with 0 object(s) at 1757442668.25
[DRAW] Frame with 0 object(s) at 1757442668.26
[DRAW] Frame with 0 object(s) at 1757442668.28
[DRAW] Frame with 0 object(s) at 1757442668.30
[DRAW] Frame with 0 object(s) at 1757442668.32
[DRAW] Frame with 0 object(s) at 1757442668.33
[DRAW] Frame with 0 object(s) at 1757442668.35
[DRAW] Frame with 0 object(s) at 1757442668.37
[DRAW] Frame with 0 object(s) at 1757442668.38
[DRAW] Frame with 0 object(s) at 1757442668.40
[DRAW] Frame with 0 o

[DRAW] Frame with 0 object(s) at 1757442746.22
[DRAW] Frame with 0 object(s) at 1757442746.23
[DRAW] Frame with 0 object(s) at 1757442746.25
[DRAW] Frame with 0 object(s) at 1757442746.27
[DRAW] Frame with 0 object(s) at 1757442746.29
[DRAW] Frame with 0 object(s) at 1757442746.30
[DRAW] Frame with 0 object(s) at 1757442746.32
[DRAW] Frame with 0 object(s) at 1757442746.34
[DRAW] Frame with 0 object(s) at 1757442746.35
[DRAW] Frame with 0 object(s) at 1757442746.37
[DRAW] Frame with 0 object(s) at 1757442746.39
[DRAW] Frame with 0 object(s) at 1757442746.40
[DRAW] Frame with 0 object(s) at 1757442746.42
[DRAW] Frame with 0 object(s) at 1757442746.44
[DRAW] Frame with 0 object(s) at 1757442746.45
[DRAW] Frame with 0 object(s) at 1757442746.47
[DRAW] Frame with 0 object(s) at 1757442746.49
[DRAW] Frame with 0 object(s) at 1757442746.50
[DRAW] Frame with 0 object(s) at 1757442746.52
[DRAW] Frame with 0 object(s) at 1757442746.54
[DRAW] Frame with 0 object(s) at 1757442746.56
[DRAW] Frame 

In [ ]:
#